In [1]:
import pybullet as p
import time
import pybullet_data
import math
import numpy as np
import matplotlib.pyplot as plt



############# hand control ################
finger_bend_angle=60





physicsClient=p.connect(p.DIRECT)
# p.setAdditionalSearchVisualizer()
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.resetSimulation(p.RESET_USE_DEFORMABLE_WORLD)

p.setGravity(0,0,-9.8) # x,y,z direction


# camera position
p.configureDebugVisualizer(p.COV_ENABLE_GUI,0)
p.resetDebugVisualizerCamera(cameraDistance=0.4,cameraYaw=50,cameraPitch=-50,cameraTargetPosition=[0,0,0])


# define start pose
StartPos_cuboid=[0.5, 0, 0.04]     #[0.1,0,0.03998089919150022] 



StartPos_sphere=[-1,0,0.049987949685306586]  
StartPos_arh,StartOrn_arh=[0,0,0],p.getQuaternionFromEuler([0,0,0])
StartOrientation=p.getQuaternionFromEuler([0,0,0])
soft_body_position= [0.09,0,0.04]

planeId=p.loadURDF("plane.urdf")
StartPos_lite6,StartOrn_lite_6=[0,0,0],p.getQuaternionFromEuler([0,0,0])
########################## Rigid Bodies ###################
cuboidId=p.loadURDF(r"models/simple_object/box.urdf",StartPos_cuboid)
sphereId=p.loadURDF(r"models/simple_object/sphere.urdf",StartPos_sphere)

lite_arm_Id=p.loadURDF(r"models/arm/lite6.urdf",StartPos_lite6,StartOrn_lite_6)
# armId=p.loadURDF(r"models/end_effector/arm.urdf",StartPos_sphere)
# arhId=p.loadURDF(r"models/end_effector/left.urdf")
lite6_end_effector_link=7
arh_base_link=0

pos_lite6_end_effector_link, orn_lite6_end_effector_link=p.getLinkState(lite_arm_Id,lite6_end_effector_link)[:2]

arhId=p.loadURDF(r"models/end_effector/left.urdf",pos_lite6_end_effector_link,orn_lite6_end_effector_link)
print("line xxxxxxxxxxxxxx")
print(pos_lite6_end_effector_link, orn_lite6_end_effector_link)
print("line xxxxxxxxxxxxxx")


pos_arh_base_link, orn_arh_base_link=p.getBasePositionAndOrientation(arhId)[:2]
print(pos_arh_base_link,orn_arh_base_link)
print("XXXXXXXXXXXXXxxxxxx")


pybullet build time: Nov 28 2023 23:45:17


b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
link_eefline xxxxxxxxxxxxxx
(0.08699860411939289, -7.134240641805314e-07, 0.15358936087493205) (0.9999999999763882, 5.509814401301733e-06, -3.673194983925448e-06, 1.8366160441929402e-06)
line xxxxxxxxxxxxxx
(0.08699860411939289, -7.134240641805314e-07, 0.15358936087493205) (0.9999999999763882, 5.509814401301733e-06, -3.673194983925448e-06, 1.8366160441929402e-06)
XXXXXXXXXXXXXxxxxxx
b3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
No inertial data for link, using mass=1, localinertiadiagonal = 1,1,1, identity local inertial frameb3Warning[examples/Importers/ImportURDFDemo/BulletUrdfImporter.cpp,126]:
hand_root

In [2]:
cube_big = p.loadSoftBody("ball.obj", simFileName = "ball.vtk", basePosition =  [-1.03,-0.03, 0.5], scale = 0.5, mass = 4, useNeoHookean = 1 , NeoHookeanMu = 700, NeoHookeanLambda = 800, NeoHookeanDamping = 0.01, useSelfCollision = 1, frictionCoeff = .5, collisionMargin = 0.01)
p.setTimeStep(1/240.)
p.changeDynamics(cube_big,-1, lateralFriction=0.5)

In [3]:
def define_bot(bot_id):
    bot_movable_joints={}
    num_joint=p.getNumJoints(bot_id)
    print(f"total number of joint {num_joint}")
    for current_joint in range(num_joint):
        botJointInfo=p.getJointInfo(
            bodyUniqueId=bot_id,
            jointIndex=current_joint)
            
        if botJointInfo[2]!=4: # non fixed joints 
            
            defaultState=p.getJointState(bot_id,current_joint)
            bot_movable_joints[current_joint]={"Name":botJointInfo[1],
                                               "index":current_joint,
                                               "type":botJointInfo[2],
                                               "jointCurrentState":defaultState[0],
                                            "jointLowerLimit":botJointInfo[8], # 8 lower limit
                                            "jointUpperLimit":botJointInfo[9]} # 9 upper limit
            print(f"movable joint {bot_movable_joints[current_joint]}")
    return bot_movable_joints

#         print(f"joint info of joint {current_joint}: \n {KukaJointInfo[2]} \n ################### end ################")

        
    

In [4]:
lite6_dict=define_bot(lite_arm_Id)

total number of joint 8
movable joint {'Name': b'joint1', 'index': 1, 'type': 0, 'jointCurrentState': 0.0, 'jointLowerLimit': -6.283185307179586, 'jointUpperLimit': 6.283185307179586}
movable joint {'Name': b'joint2', 'index': 2, 'type': 0, 'jointCurrentState': 0.0, 'jointLowerLimit': -2.61799, 'jointUpperLimit': 2.61799}
movable joint {'Name': b'joint3', 'index': 3, 'type': 0, 'jointCurrentState': 0.0, 'jointLowerLimit': -0.061087, 'jointUpperLimit': 5.235988}
movable joint {'Name': b'joint4', 'index': 4, 'type': 0, 'jointCurrentState': 0.0, 'jointLowerLimit': -6.283185307179586, 'jointUpperLimit': 6.283185307179586}
movable joint {'Name': b'joint5', 'index': 5, 'type': 0, 'jointCurrentState': 0.0, 'jointLowerLimit': -2.1642, 'jointUpperLimit': 2.1642}
movable joint {'Name': b'joint6', 'index': 6, 'type': 0, 'jointCurrentState': 0.0, 'jointLowerLimit': -6.283185307179586, 'jointUpperLimit': 6.283185307179586}


In [5]:
def simulation_fun(bot_id,link):
    steps=50
    print("running simulation block")
    for c_step in range(steps):
         p.stepSimulation()
         time.sleep(1./24.)
         current_positino=p.getJointState(
             bodyUniqueId=bot_id,
             jointIndex=link)   
         print(f"current position: {math.degrees(current_positino[0])}")
    

In [6]:
def bot_mover_demo(bot_id):
    bot_dict=define_bot(bot_id)
    for key in bot_dict.keys():
        print(key) 
        target_state=bot_dict[key]["jointLowerLimit"]
        p.setJointMotorControlArray(
            bodyUniqueId=bot_id, 
            jointIndices=[key], 
            controlMode=p.POSITION_CONTROL,  
            targetPositions=[bot_dict[key]["jointLowerLimit"]]
        )
        print(f"moving joint {key} to lower limit {math.degrees(bot_dict[key]['jointLowerLimit'])}")
        time.sleep(1)
        simulation_fun(bot_id,key)
        print("simulation done")
        time.sleep(4)
        print("going back to original position")
        p.setJointMotorControlArray(
            bodyUniqueId=bot_id, 
            jointIndices=[key], 
            controlMode=p.POSITION_CONTROL,  
            targetPositions=[bot_dict[key]["jointCurrentState"]]
        )
        print(f"moving joint {key} to default state {math.degrees(bot_dict[key]['jointCurrentState'])}")
        time.sleep(1)
        simulation_fun(bot_id,key)
        print("simulation done")
        time.sleep(4)
        


In [7]:
bot_mover_demo(lite_arm_Id)

total number of joint 8
movable joint {'Name': b'joint1', 'index': 1, 'type': 0, 'jointCurrentState': 0.0, 'jointLowerLimit': -6.283185307179586, 'jointUpperLimit': 6.283185307179586}
movable joint {'Name': b'joint2', 'index': 2, 'type': 0, 'jointCurrentState': 0.0, 'jointLowerLimit': -2.61799, 'jointUpperLimit': 2.61799}
movable joint {'Name': b'joint3', 'index': 3, 'type': 0, 'jointCurrentState': 0.0, 'jointLowerLimit': -0.061087, 'jointUpperLimit': 5.235988}
movable joint {'Name': b'joint4', 'index': 4, 'type': 0, 'jointCurrentState': 0.0, 'jointLowerLimit': -6.283185307179586, 'jointUpperLimit': 6.283185307179586}
movable joint {'Name': b'joint5', 'index': 5, 'type': 0, 'jointCurrentState': 0.0, 'jointLowerLimit': -2.1642, 'jointUpperLimit': 2.1642}
movable joint {'Name': b'joint6', 'index': 6, 'type': 0, 'jointCurrentState': 0.0, 'jointLowerLimit': -6.283185307179586, 'jointUpperLimit': 6.283185307179586}
1
moving joint 1 to lower limit -360.0
running simulation block
current posi